**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [10]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-o_673ifr
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-o_673ifr
  fatal: unable to access 'https://github.com/Kaggle/learntools.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-o_673ifr did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-o_673ifr did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [11]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,ca2943325bb86524ccb494a82c4361e2e59d94b9,4ab868a2a19b95f50f355eb8d0d7638f1912b37f49ebd6...,2013-04-14 14:45:00+00:00,2013-04-14 15:00:00+00:00,660,0.0,NaN,NaN,14,14,...,0.0,6.85,Cash,Blue Ribbon Taxi Association Inc.,41.968069,-87.721559,POINT (-87.7215590627 41.968069),41.968069,-87.721559,POINT (-87.7215590627 41.968069)
1,65772aeff3599d099f040068b9b7cd4491aea5b6,6682944f2b37609761181033253037c05fed4377e8b649...,2013-05-12 20:15:00+00:00,2013-05-12 20:30:00+00:00,240,0.9,NaN,NaN,41,41,...,1.0,6.05,Cash,Northwest Management LLC,41.794090,-87.592311,POINT (-87.592310855 41.794090253),41.794090,-87.592311,POINT (-87.592310855 41.794090253)
2,be9aee48bd3bc2e919ad76956c94e0bc420366f1,af5b394cb22984f667fc0a9cc30e0277cb9edf50f14747...,2013-04-17 01:45:00+00:00,2013-04-17 02:00:00+00:00,780,0.0,1.703184e+10,1.703124e+10,28,24,...,1.5,12.95,Cash,Dispatch Taxi Affiliation,41.867902,-87.642959,POINT (-87.6429586652 41.8679024175),41.906026,-87.675312,POINT (-87.6753116216 41.906025969)
3,e1a250fcf325056fd51fe87e655f2ee15acd4599,4de82fc17cb23f745931085fbbb3b4f920550cda576979...,2013-04-14 14:30:00+00:00,2013-04-14 14:45:00+00:00,360,1.2,1.703107e+10,1.703183e+10,7,6,...,0.0,6.05,Cash,Dispatch Taxi Affiliation,41.930579,-87.642206,POINT (-87.6422063127 41.9305785697),41.946295,-87.654298,POINT (-87.6542980837 41.9462945357)
4,40c021586099a8bb48795aebbb601384304eb60e,486a31c847d3de20267c610f8de52bdd13c696a8a22bcb...,2013-04-21 02:15:00+00:00,2013-04-21 02:15:00+00:00,360,0.0,1.703107e+10,1.703107e+10,7,7,...,2.0,7.25,Cash,Dispatch Taxi Affiliation,41.929263,-87.635891,POINT (-87.6358909539 41.9292629902),41.921855,-87.646211,POINT (-87.6462109769 41.9218549112)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [12]:
# 택시 수요를 예측하기 위해 매일 택시 여행의 롤링 평균을 보여주는 그래프를 생성

# trip_date - 2016년 1월 1일부터 2016년 3월 31일까지 각 날짜별로 하나의 항목이 포함
# avg_num_trips - 현재 날짜의 값을 포함하며, 
# 이전 3일과 다음 3일 동안의 택시 여행 수를 포함하는 창을 사용하여 계산된 평균 일일 여행 수


# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """
# AVG(num_trips) OVER (...) 구문: 윈도우 함수를 사용하여 롤링 평균을 계산하는 방법
# 1. trip_date 기준으로 정렬
# 2. ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
# : 이 구문은 현재 행을 포함하여 이전 3개 행과 다음 3개 행을 포함하는 윈도우를 지정

# 롤링평균 : 
# 특정 테이블에서 데이터를 조회하여 각 날짜별로 평균 택시 여행 수를 계산하는 작업 수행
# trip_date: 이 컬럼은 각 날짜별로 택시 여행 수
# trips_by_day 테이블에서 이 컬럼은 각 날짜에 대한 정보
# OVER (...): 여기서 ... 부분은 윈도우 명세를 정의
# 특정 날짜를 기준으로 이전 3일과 다음 3일 동안의 데이터를 포함하여 평균을 계산

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-02-27,92965.142857
1,2016-03-12,99863.428571
2,2016-02-09,87852.428571
3,2016-01-27,81378.428571
4,2016-02-28,92816.428571


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [13]:
# Lines below will give you a hint or solution code
#q_1.hint()
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

```

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [19]:
# pickup_community_area, trip_start_timestamp, trip_end_timestamp >> 열 반환

# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """
# PARTITION BY 절은 데이터를 파티션으로 나누는 역할 >> 출발 지역
# ORDER BY 절은 RANK() 함수에 의해 순위를 매길 기준이 되는 열을 지정
# >>  각 택시 출발 지역 내에서 여행이 시작된 순서대로 순위

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,25.0,2013-10-03 00:30:00+00:00,2013-10-03 00:30:00+00:00,1
1,25.0,2013-10-03 02:45:00+00:00,2013-10-03 03:00:00+00:00,2
2,25.0,2013-10-03 04:45:00+00:00,2013-10-03 05:15:00+00:00,3
3,25.0,2013-10-03 06:15:00+00:00,2013-10-03 06:45:00+00:00,4
4,25.0,2013-10-03 06:45:00+00:00,2013-10-03 07:00:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [18]:
# Lines below will give you a hint or solution code
#q_2.hint()
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

```

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [16]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) 
                           OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE
                           ) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,01271057890af916c8ee4890c076e44fb6ac327a1868de...,2013-10-03 16:30:00+00:00,2013-10-03 16:45:00+00:00,270.0
1,01271057890af916c8ee4890c076e44fb6ac327a1868de...,2013-10-03 23:30:00+00:00,2013-10-03 23:15:00+00:00,195.0
2,0e459cbb5049e55828a11cd1deb08a2d25359dfd6f1a77...,2013-10-03 05:45:00+00:00,2013-10-03 06:00:00+00:00,270.0
3,0e459cbb5049e55828a11cd1deb08a2d25359dfd6f1a77...,2013-10-03 13:30:00+00:00,2013-10-03 13:30:00+00:00,285.0
4,10e8a565ac8829bd9caa44f57e9831fd8b4ff374e4d666...,2013-10-03 14:00:00+00:00,2013-10-03 13:45:00+00:00,825.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [17]:
# Lines below will give you a hint or solution code
#q_3.hint()
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

break_time_result = client.query(break_time_query).result().to_dataframe()

```

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*